In [97]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix

# Load stock data (example: CSV file with historical stock data)
# The dataset should have features like 'Open', 'High', 'Low', 'Close', 'Volume', etc.
dataGold = pd.read_csv('./CSV/df_gold.csv')
dataSP500 = pd.read_csv('./CSV/df_sp500.csv')
dataBTC = pd.read_csv('./CSV/df_btc.csv')

# 365 première valeurs
dataGold = dataGold.head(365)
dataSP500 = dataSP500.head(365)
dataBTC = dataBTC.head(365)

#Create target variable: 1 if the price will go up, 0 if it will go down
dataGold['Price_Up'] = np.where(dataGold['Close/Last'].shift(-1) > dataGold['Close/Last'], 1, 0)

dataSP500['Price_Up'] = np.where(dataSP500['Close/Last'].shift(-1) > dataSP500['Close/Last'], 1, 0)

dataBTC['Price_Up'] = np.where(dataBTC['Close/Last'].shift(-1) > dataBTC['Close/Last'], 1, 0)

#Date,Close/Last,High,Low,Open,Volume
# Select features for the model (e.g., 'Open', 'High', 'Low', 'Close', 'Volume')
features = ['Close/Last', 'Open']
Xgold = dataGold[features]
ygold = dataGold['Price_Up']

Xsp500 = dataSP500[features]
ysp500 = dataSP500['Price_Up']

Xbtc = dataBTC[features]
ybtc = dataBTC['Price_Up']

X_train_gold, X_test_gold, y_train_gold, y_test_gold = train_test_split(Xgold, ygold, test_size=0.2, random_state=42)

X_train_sp500, X_test_sp500, y_train_sp500, y_test_sp500 = train_test_split(Xsp500, ysp500, test_size=0.2, random_state=42)

X_train_btc, X_test_btc, y_train_btc, y_test_btc = train_test_split(Xbtc, ybtc, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()

X_train_scaled_gold = scaler.fit_transform(X_train_gold)
X_test_scaled_gold = scaler.transform(X_test_gold)

X_train_scaled_sp500 = scaler.fit_transform(X_train_sp500)
X_test_scaled_sp500 = scaler.transform(X_test_sp500)

X_train_scaled_btc = scaler.fit_transform(X_train_btc)
X_test_scaled_btc = scaler.transform(X_test_btc)

# Train the logistic regression model
model_gold = LogisticRegression()
model_sp500 = LogisticRegression()
model_btc = LogisticRegression()

model_gold.fit(X_train_scaled_gold, y_train_gold)
model_sp500.fit(X_train_scaled_sp500, y_train_sp500)
model_btc.fit(X_train_scaled_btc, y_train_btc)


# Make predictions on the test set
y_pred_gold = model_gold.predict(X_test_scaled_gold)
y_pred_sp500 = model_sp500.predict(X_test_scaled_sp500)
y_pred_btc = model_btc.predict(X_test_scaled_btc)

# Evaluate the model
accuracy_gold = accuracy_score(y_test_gold, y_pred_gold)
conf_matrix_gold = confusion_matrix(y_test_gold, y_pred_gold)

accuracy_sp500 = accuracy_score(y_test_sp500, y_pred_sp500)
conf_matrix_sp500 = confusion_matrix(y_test_sp500, y_pred_sp500)

accuracy_btc = accuracy_score(y_test_btc, y_pred_btc)
conf_matrix_btc = confusion_matrix(y_test_btc, y_pred_btc)


# Display results
print(f'Accuracy: {accuracy_gold * 100:.2f}%')
print('Confusion Matrix:')
print(conf_matrix_gold)

print(f'Accuracy: {accuracy_sp500 * 100:.2f}%')
print('Confusion Matrix:')
print(conf_matrix_sp500)

print(f'Accuracy: {accuracy_btc * 100:.2f}%')
print('Confusion Matrix:')
print(conf_matrix_btc)



Accuracy: 60.27%
Confusion Matrix:
[[ 0 29]
 [ 0 44]]
Accuracy: 50.68%
Confusion Matrix:
[[ 0 36]
 [ 0 37]]
Accuracy: 53.42%
Confusion Matrix:
[[ 0 34]
 [ 0 39]]
